# Toronto Neighborhood Clustering


In [1]:
#importing modules
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
!conda install -c conda-forge folium=0.5.0
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 331.62 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 381.59 kB/s
altair-2.2.2-p 100% |################################| Time: 0:00:00 545.46 kB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  37.59 MB/s


In [2]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [3]:
toronto_neighbor_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup = BeautifulSoup(toronto_neighbor_page, 'html')
#print(soup.prettify())

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/envs/DSX-Python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [5]:
mytable = soup.find('table')

In [6]:
cols = mytable.find_all('td')

In [7]:
# creating a list of postal codes, borough and neighborhood by scraping using 
postal_code = []
borough = []
neighborhood = []
for info in cols:
    if info.find('span').find('i'):
        continue
    else:
        postal_code.append(info.find('b').text)
        text = info.find('span').text
        idx = text.find('(')
        if idx != -1:
            borough.append(text[:idx])
            neighborhood.append(text[idx+1: ])
        else:
            borough.append('')
            neighborhood.append('')
#print(postal_code, borough, neighborhood)
neighborhoods = []
for neighbor in neighborhood:
    neighborhoods.append(neighbor.replace('/', ',').replace(')', ' ').replace('(', ',').rstrip())
#print("nrighbor = ", neighborhoods)
#print(len(postal_code), len(borough), len(neighborhoods))


In [8]:
# PostalCode, Borough, and Neighborhood
df = pd.DataFrame({'PostalCode':postal_code, 'Borough':borough, 'Neighborhood': neighborhoods})
df

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park , Harbourfront",M5A
3,North York,"Lawrence Manor , Lawrence Heights",M6A
4,,,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Malvern , Rouge",M1B
7,North York,Don Mills North,M3B
8,East York,"Parkview Hill , Woodbine Gardens",M4B
9,Downtown Toronto,"Garden District, Ryerson",M5B


In [9]:
#getting values of dataframe columns to pass in the google's geocoding api
postal_codes = df['PostalCode'].get_values()
neighborhood = df['Neighborhood'].get_values()
#print(postal_codes, len(postal_codes))
#print(neighborhood, len(neighborhood))


In [16]:
# The code was removed by Watson Studio for sharing.

In [18]:
# Getting the latitude and longitude using google geocoding api 
locations = []
# construct URL to make API call
for idx in range(len(postal_codes)):
    #print(postal_codes[idx] + "," + neighborhood[idx])
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, postal_codes[idx] + "," + neighborhood[idx])
    response = requests.get(url).json() # get response
    #print(response)
    if len(response['results']) > 0:
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        latitude = geographical_data['lat']
        longitude = geographical_data['lng']
        #print(latitude, longitude)
        locations.append((latitude, longitude))
    else:
        latitude = ''
        longitude = ''
        locations.append((latitude, longitude))
        continue
print(locations)

[(43.7532586, -79.3296565), (43.72588229999999, -79.3155716), (43.638834, -79.3818785), (43.72277400000001, -79.45092799999999), ('', ''), (43.6619767, -79.5309608), (43.8066863, -79.1943534), (43.7459058, -79.352188), (43.7061385, -79.3254151), (43.6576585, -79.3788017), (43.7087787, -79.4407294), (43.7213234, -79.5845715), (43.78453510000001, -79.1604971), (43.7211861, -79.3389695), (43.693328, -79.316064), (43.6708666, -79.37330639999999), (43.6937813, -79.4281914), (43.643238, -79.580168), (43.7928371, -79.19593499999999), (43.6763574, -79.2930312), (43.6479848, -79.37522530000001), (43.6869159, -79.45632499999999), (43.767528, -79.21772100000001), (43.7090604, -79.3634517), (43.6561311, -79.3842646), (43.6727456, -79.4219686), (43.773136, -79.23947609999999), (43.8037622, -79.3634517), (43.7481449, -79.45464919999999), (43.7053689, -79.3493719), (43.6496814, -79.3837757), (43.8446696, -79.48354859999999), (43.7447342, -79.23947609999999), (43.77178199999999, -79.352086), (43.77345

In [19]:
lats = []
longs = []
for location in locations:
    lat = location[0]
    long = location[1]
    lats.append(lat)
    longs.append(long)
df['Latitude'] = lats
df['Longitude'] = longs
df.loc[89, ["Latitude"]] = 43.7394164
df.loc[89, "Longitude"] = -79.58843689999999
df.loc[76, ["Latitude"]] = 43.63827828029151
df.loc[76, "Longitude"] = -79.61459401970849
df.loc[70, ["Latitude"]] = 43.696319
df.loc[76, "Longitude"] = -79.5322424
df.shape

(103, 5)

In [20]:
df.tail()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
98,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",M8X,43.6515,-79.4938
99,Downtown Toronto,Church and Wellesley,M4Y,43.6657,-79.3809
100,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,M7Y,43.6627,-79.3216
101,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",M8Y,43.6252,-79.5076
102,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",M8Z,43.6486,-79.5053


In [21]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', '', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'], dtype=object)

In [27]:
toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto.shape

(39, 5)

In [23]:
url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, 'Toronto, Canada')
response = requests.get(url).json() # get response
geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
latitude = geographical_data['lat']
longitude = geographical_data['lng']
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653226, -79.3831843.


In [24]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [32]:
CLIENT_ID = 'TABHGZSJZA3LYUX3HJXSNNDTEDWH5K2TP2UV4YMZME0OFYGM' # your Foursquare ID
CLIENT_SECRET = '14TTXYGOOXBTCC5NMOO2JG4OAB502JQBEMHAZQGJ2IERKER1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TABHGZSJZA3LYUX3HJXSNNDTEDWH5K2TP2UV4YMZME0OFYGM
CLIENT_SECRET:14TTXYGOOXBTCC5NMOO2JG4OAB502JQBEMHAZQGJ2IERKER1


#### Let's explore the first neighborhood in our dataframe.

In [33]:
toronto.loc[0, 'Neighborhood']

'Regent Park , Harbourfront'

In [34]:
neighborhood_latitude = toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park , Harbourfront are 43.638834, -79.3818785.


#### Now, let's get the top 100 venues that are in Regent Park within a radius of 500 meters.

In [35]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b7e470ddb04f5262685ff57'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad4c05ef964a52094f620e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f2931735',
         'name': 'Performing Arts Venue',
         'pluralName': 'Performing Arts Venues',
         'primary': True,
         'shortName': 'Performing Arts'}],
       'id': '4ad4c05ef964a52094f620e3',
       'location': {'address': '235 Queens Quay West',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Lower Simcoe St.',
        'distance': 110,
        'formattedAddress': ['235 Queens Quay West (at Lower Simcoe St.)',
        

In [37]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

(100, 4)

In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                )
toronto_venues.shape

Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
The Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Enclave of M5E
St. Jame

NameError: name 'toronoto_venues' is not defined

In [43]:
toronto_venues.shape

(1777, 7)

In [44]:
#counting how many results came for each venue
toronto_venues.groupby('Neighborhood').count().shape

(39, 6)

In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 243 uniques categories.


## 3. Analyze Each Neighborhood

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(1777, 243)

In [47]:
toronto_onehot.shape

(1777, 243)

In [49]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(39, 243)

In [50]:
toronto_grouped.shape

(39, 243)

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2       Bakery  0.04
3   Restaurant  0.04
4        Hotel  0.04


----Brockton , Parkdale Village , Exhibition Place----
                venue  freq
0                Park  0.19
1  Athletics & Sports  0.12
2      Soccer Stadium  0.12
3           Racetrack  0.06
4                Café  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                venue  freq
0      Airport Lounge  0.18
1     Airport Service  0.18
2    Airport Terminal  0.18
3             Airport  0.09
4  Airport Food Court  0.09


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.10
1  Clothing Store  0.08
2            Café  0.04
3           Plaza  0.03
4  Cosmetics Shop  0.03


----Christie----
                venue  freq
0       Grocery Store  0.15
1                Park  0.15
2         Coffee Shop  0.08
3         Candy Store  0.08
4  Ita

###### First, let's write a function to sort the venues in descending order.

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Bakery,Italian Restaurant,Restaurant,Hotel,Gastropub,Japanese Restaurant,Beer Bar,Cocktail Bar
1,"Brockton , Parkdale Village , Exhibition Place",Park,Soccer Stadium,Athletics & Sports,Train Station,Farm,Café,Arts & Crafts Store,Theme Park,Coffee Shop,Hotel
2,"CN Tower , King and Spadina , Railway Lands , ...",Airport Terminal,Airport Service,Airport Lounge,Boutique,Airport Gate,Plane,Airport Food Court,Airport,Electronics Store,Flower Shop
3,Central Bay Street,Coffee Shop,Clothing Store,Café,Italian Restaurant,Plaza,Cosmetics Shop,Tea Room,Thai Restaurant,Japanese Restaurant,Bubble Tea Shop
4,Christie,Park,Grocery Store,Japanese Restaurant,Italian Restaurant,Café,Coffee Shop,Beer Store,Nightclub,Candy Store,Gym
5,Church and Wellesley,Coffee Shop,Gay Bar,Burger Joint,Japanese Restaurant,Men's Store,Breakfast Spot,Bubble Tea Shop,Restaurant,Ramen Restaurant,Nightclub
6,"Commerce Court , Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,American Restaurant,Deli / Bodega,Gym,Cocktail Bar,Gastropub
7,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Seafood Restaurant,Café,Sushi Restaurant,Italian Restaurant,Coffee Shop,Pharmacy,Thai Restaurant
8,Davisville North,Hotel,Burger Joint,Electronics Store,Park,Grocery Store,Sandwich Place,Dance Studio,Food & Drink Shop,Breakfast Spot,Fast Food Restaurant
9,"Dufferin , Dovercourt Village",Pharmacy,Grocery Store,Bus Stop,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm


## 4. Cluster Neighborhoods


Run *k*-means to cluster the neighborhood into 5 clusters.

In [54]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
#print(toronto_grouped_clustering)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)
neighborhoods_venues_sorted.shape
#print(kmeans.labels_)
toronto_grouped_clustering

toronto_grouped['Neighborhood']

0                                           Berczy Park
1        Brockton , Parkdale Village , Exhibition Place
2     CN Tower , King and Spadina , Railway Lands , ...
3                                    Central Bay Street
4                                              Christie
5                                  Church and Wellesley
6                       Commerce Court , Victoria Hotel
7                                            Davisville
8                                      Davisville North
9                         Dufferin , Dovercourt Village
10                                       Enclave of M4L
11                                       Enclave of M5E
12              First Canadian Place , Underground city
13                             Forest Hill North & West
14                             Garden District, Ryerson
15    Harbourfront East , Union Station , Toronto Is...
16                       High Park , The Junction South
17                      India Bazaar , The Beach

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [55]:

toronto['Neighborhood']

0                            Regent Park , Harbourfront
1                              Garden District, Ryerson
2                                        St. James Town
3                                           The Beaches
4                                           Berczy Park
5                                    Central Bay Street
6                                              Christie
7                            Richmond , Adelaide , King
8                         Dufferin , Dovercourt Village
9                                    The Danforth  East
10    Harbourfront East , Union Station , Toronto Is...
11                            Little Portugal , Trinity
12                        The Danforth West , Riverdale
13            Toronto Dominion Centre , Design Exchange
14       Brockton , Parkdale Village , Exhibition Place
15                      India Bazaar , The Beaches West
16                      Commerce Court , Victoria Hotel
17                                      Studio D

In [56]:
toronto_merged = toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.6388,-79.3819,1,Coffee Shop,Boat or Ferry,Pizza Place,Café,Hotel,Bar,Sports Bar,Fast Food Restaurant,Park,Brewery
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.6577,-79.3788,0,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Cosmetics Shop,Ramen Restaurant,Falafel Restaurant,Middle Eastern Restaurant
2,Downtown Toronto,St. James Town,M5C,43.6709,-79.3733,1,Coffee Shop,Pizza Place,Hotel,Bakery,Bar,Breakfast Spot,Smoothie Shop,Sandwich Place,Caribbean Restaurant,Restaurant
3,East Toronto,The Beaches,M4E,43.6764,-79.293,1,Pub,Music Venue,Coffee Shop,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm
4,Downtown Toronto,Berczy Park,M5E,43.648,-79.3752,0,Coffee Shop,Café,Bakery,Italian Restaurant,Restaurant,Hotel,Gastropub,Japanese Restaurant,Beer Bar,Cocktail Bar


In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Cosmetics Shop,Ramen Restaurant,Falafel Restaurant,Middle Eastern Restaurant
4,Berczy Park,0,Coffee Shop,Café,Bakery,Italian Restaurant,Restaurant,Hotel,Gastropub,Japanese Restaurant,Beer Bar,Cocktail Bar
35,"St. James Town , Cabbagetown",0,Restaurant,Café,Coffee Shop,Grocery Store,Indian Restaurant,Chinese Restaurant,Pub,Diner,Japanese Restaurant,Italian Restaurant
38,Enclave of M4L,0,Pizza Place,Recording Studio,Fast Food Restaurant,Smoke Shop,Skate Park,Spa,Burrito Place,Light Rail Station,Auto Workshop,Farmers Market


In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park , Harbourfront",1,Coffee Shop,Boat or Ferry,Pizza Place,Café,Hotel,Bar,Sports Bar,Fast Food Restaurant,Park,Brewery
2,St. James Town,1,Coffee Shop,Pizza Place,Hotel,Bakery,Bar,Breakfast Spot,Smoothie Shop,Sandwich Place,Caribbean Restaurant,Restaurant
3,The Beaches,1,Pub,Music Venue,Coffee Shop,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm
5,Central Bay Street,1,Coffee Shop,Clothing Store,Café,Italian Restaurant,Plaza,Cosmetics Shop,Tea Room,Thai Restaurant,Japanese Restaurant,Bubble Tea Shop
6,Christie,1,Park,Grocery Store,Japanese Restaurant,Italian Restaurant,Café,Coffee Shop,Beer Store,Nightclub,Candy Store,Gym
7,"Richmond , Adelaide , King",1,Café,Coffee Shop,American Restaurant,Steakhouse,Thai Restaurant,Japanese Restaurant,Deli / Bodega,Restaurant,Burger Joint,Asian Restaurant
8,"Dufferin , Dovercourt Village",1,Pharmacy,Grocery Store,Bus Stop,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm
10,"Harbourfront East , Union Station , Toronto Is...",1,Boat or Ferry,Coffee Shop,Pizza Place,Sporting Goods Shop,Liquor Store,Sushi Restaurant,Hotel,Bar,Basketball Stadium,Steakhouse
11,"Little Portugal , Trinity",1,Bar,Café,Coffee Shop,Restaurant,Bakery,French Restaurant,Vietnamese Restaurant,Asian Restaurant,Pizza Place,Men's Store
12,"The Danforth West , Riverdale",1,Greek Restaurant,Coffee Shop,Sushi Restaurant,Pub,Pizza Place,Ice Cream Shop,Breakfast Spot,Italian Restaurant,Bookstore,Diner


The clustering with the toronto data I understand that almost all locations most common venues have coffee shops, cafes, restaurants but it all comes to  among last common venues where they have been divided into clusters so evervy area have consistent venues for eating, drinking places first which are visited and then comes the other places.